In [1]:
from PIL import Image
import itertools
import numpy as np
import pandas as pd
import warnings
import nltk
import math
import time
import re
import os
import pickle
#Deep-Learning Library
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = "/content/drive/My Drive/Major Project B.Tech Sem 8/Breast Cancer Dataset/"
savingpath="/content/drive/My Drive/Major Project B.Tech Sem 8/"

In [0]:
train_data_dir=path+"Train"
test_data_dir=path+"Test"

#Training paths for saving
data_cnn_Xception_train=savingpath+"Xception/train/data_cnn_Xception_train.npy"
data_filenames_Xception_train=savingpath+"Xception/train/data_filenames_Xception_train.npy"
data_cancerclass_Xception_train=savingpath+"Xception/train/data_cancerclass_Xception_train.npy"
data_cancertype_Xception_train=savingpath+"Xception/train/data_cancertype_Xception_train.npy"
data_mag_Xception_train=savingpath+"Xception/train/data_mag_Xception_train.npy"

#testing paths for saving
data_cnn_Xception_test=savingpath+"Xception/test/data_cnn_Xception_test.npy"
data_filenames_Xception_test=savingpath+"Xception/test/data_filenames_Xception_test.npy"
data_cancerclass_Xception_test=savingpath+"Xception/test/data_cancerclass_Xception_test.npy"
data_cancertype_Xception_test=savingpath+"Xception/test/data_cancertype_Xception_test.npy"
data_mag_Xception_test=savingpath+"Xception/test/data_mag_Xception_test.npy"

In [6]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 299, 299
epochs = 50
batch_size = 1
print("Started")

def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return "NO" 
    else: 
        return "YES" 

def save_bottlebeck_features_train(train_data=train_data_dir,nb_train_samples=5000,batch_size=1):
    
    #Function to compute Xception CNN for image feature extraction.
        
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the VGG19 network
    model = applications.Xception(include_top=False, weights='imagenet')
    print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    for i in generator.filenames:
        
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        k=0
        for types in cancersubtypes:          
          flagforcancersubtypes=check(i,types)         
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(flagforcancersubtypes=="YES"):
            cancertype.append(codeforcancersubtypes[k])
          k=k+1
         
        # Array for Magnification
        magnificationsformat=["-40-","-100-","-200-","-400-"]
        codeformagnifications=[40,100,200,400]
        j=0
        for types in magnificationsformat:
          
          flagformagnifications=check(i,types)         
          if(flagformagnifications=="YES"):
            mag.append(codeformagnifications[j])
          j=j+1
        
    print(cancerclass)    
    print(cancertype)
    print(mag)
    print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    print("training start")
    start_time = time.clock()
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    print(time.clock() - start_time, "seconds, time for training")
    print("training finised")
    print("reshape the trained vector")
    bottleneck_features_train = bottleneck_features_train.reshape((5000,204800))
    print("Trainning done")
    np.save(open(data_cnn_Xception_train, 'wb'), bottleneck_features_train)
    np.save(open(data_filenames_Xception_train, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_Xception_train, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_Xception_train, 'wb'), np.array(cancertype))
    np.save(open(data_mag_Xception_train, 'wb'), np.array(mag))
    print("npy file saved")
    

save_bottlebeck_features_train()

print(time.clock() - start_time, "seconds")

Started
Initialized Required data and datagen done
Model settled
Found 5000 images belonging to 1 classes.
Genrator work finish
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [7]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 299, 299
epochs = 50
batch_size = 1
print("Started")

def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return "NO" 
    else: 
        return "YES" 

def save_bottlebeck_features_test(train_data=test_data_dir,nb_train_samples=2900,batch_size=1):
    
    #Function to compute Xception CNN for image feature extraction.
        
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the VGG16 network
    model = applications.Xception(include_top=False, weights='imagenet')
    print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    for i in generator.filenames:
        
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        k=0
        for types in cancersubtypes:          
          flagforcancersubtypes=check(i,types)         
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(flagforcancersubtypes=="YES"):
            cancertype.append(codeforcancersubtypes[k])
          k=k+1
         
        # Array for Magnification
        magnificationsformat=["-40-","-100-","-200-","-400-"]
        codeformagnifications=[40,100,200,400]
        j=0
        for types in magnificationsformat:
          
          flagformagnifications=check(i,types)         
          if(flagformagnifications=="YES"):
            mag.append(codeformagnifications[j])
          j=j+1
        
    print(cancerclass)    
    print(cancertype)
    print(mag)
    print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    print("training start")
    start_time = time.clock()
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    print(time.clock() - start_time, "seconds, time for training")
    print("training finised")
    print("reshape the trained vector")
    bottleneck_features_train = bottleneck_features_train.reshape((2900,204800))
    print("Trainning done")
    np.save(open(data_cnn_Xception_test, 'wb'), bottleneck_features_train)
    np.save(open(data_filenames_Xception_test, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_Xception_test, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_Xception_test, 'wb'), np.array(cancertype))
    np.save(open(data_mag_Xception_test, 'wb'), np.array(mag))
    print("npy file saved")
    

save_bottlebeck_features_test()

print(time.clock() - start_time, "seconds , Testing data also saved")

Started
Initialized Required data and datagen done
Model settled
Found 2900 images belonging to 1 classes.
Genrator work finish
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1